# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [19]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [20]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


In [21]:
%%capture --no-display
### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.
# Configure the map plot

my_map = city_data_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo=True, 
    alpha=0.7, 
    tiles=True, 
    ylim=(-75, 85), 
    size='Humidity',
    color='City',
    frame_width = 600,
    frame_height = 400,
    scale=0.8)
# Display the map
my_map

ImportError: Geographic projection support requires GeoViews, pyproj and cartopy.

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [22]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_city_data = city_data_df.loc[(city_data_df['Max Temp'] < 28) & 
                                      (city_data_df['Max Temp'] > 20) &
                                      (city_data_df['Wind Speed'] < 9) &
                                      (city_data_df['Humidity'] < 40) & 
                                      (city_data_df['Cloudiness'] <= 20) 
                                      ]
# Drop any rows with null values
filtered_city_data = filtered_city_data.dropna()

# Display sample data
filtered_city_data

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
73,73,nalut,30.3333,10.8500,25.41,27,0,5.11,LY,1666108275
258,258,comodoro rivadavia,-45.8667,-67.5000,25.92,24,0,7.20,AR,1666108389
436,436,lakki marwat,32.6079,70.9114,27.89,20,0,1.53,PK,1666108492
550,550,pali,25.7667,73.3333,27.18,26,0,1.38,IN,1666108552


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_city_data.copy().reset_index()[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''


# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,nalut,LY,30.3333,10.8500,27,
1,comodoro rivadavia,AR,-45.8667,-67.5000,24,
2,lakki marwat,PK,32.6079,70.9114,20,
3,pali,IN,25.7667,73.3333,26,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [25]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel',
    'radius':radius,
    'limit': 1,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
        
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
nalut - nearest hotel: No hotel found
comodoro rivadavia - nearest hotel: No hotel found
lakki marwat - nearest hotel: No hotel found
pali - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
0,nalut,LY,30.3333,10.8500,27,No hotel found
1,comodoro rivadavia,AR,-45.8667,-67.5000,24,No hotel found
2,lakki marwat,PK,32.6079,70.9114,20,No hotel found
3,pali,IN,25.7667,73.3333,26,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [26]:
%%capture --no-display

# Configure the map plot
vaca_map = hotel_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo=True, 
    alpha=0.7, 
    tiles=True, 
    ylim=(-75, 85), 
    xlim=(-180,180),
    frame_width = 600,
    frame_height = 400, 
    size='Humidity',
    color='City',
    scale=2,
    hover_cols=['Country', 'Hotel Name'])

# Display the map
vaca_map

ImportError: Geographic projection support requires GeoViews, pyproj and cartopy.